Run this script to combine Land Use (2020), Zoning, and Building Permit Data to identify progress and remaining development potential in developing subdivisions

Data sources:

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Land_Use_2020/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Building_Permit_Records/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Planimetrics/FeatureServer

https://gis2.lawrenceks.org/ArcGIS/rest/services/Parcels/MapServer

https://tiles.arcgis.com/tiles/8O9UlSTnqjKptoda/arcgis/rest/services/LAWRENCE_KS_AERIAL_2020/MapServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Lawrence_Zoning/FeatureServer

https://services.arcgis.com/8O9UlSTnqjKptoda/arcgis/rest/services/Plats/FeatureServer



In [ ]:
filegdb = r"C:\Users\kgonterwitz\Documents\ArcGIS\Projects\Kyle Project 2\BuildingSum20210402.gdb"
arcpy.conversion.FeatureClassToFeatureClass("Plats", filegdb, "PlatsCopy", "#", '#', '')
arcpy.management.AddField(filegdb+r"\PlatsCopy", "LotBlock", "TEXT", None, None, 24, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(filegdb+r"\PlatsCopy", "LotBlock", 'str(!BLOCK!)+"-"+ str(!LOT!)', "PYTHON3", '', "TEXT")
arcpy.management.FeatureToPoint("PlatsCopy", filegdb+r"\PlatsCopyPoints", "INSIDE")
arcpy.management.FeatureToPoint("Parcels", filegdb+r"\ParcelPoints", "INSIDE")
arcpy.analysis.SummarizeWithin("LU2020", "BuildingFootprints", filegdb+r"\LandUseBuildings", "KEEP_ALL", "Shape_Area Sum", "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.analysis.SpatialJoin(filegdb+r"\LandUseBuildings",filegdb+r"\NewPermitsResidential", filegdb+r"\LandUseBuildingsPermits", "JOIN_ONE_TO_ONE", "KEEP_ALL", '#', "INTERSECT", None, '')
arcpy.management.FeatureToPoint("LandUseBuildingsPermits", filegdb+r"\LandUsePoints", "INSIDE")
arcpy.conversion.FeatureClassToFeatureClass("Building Permit", filegdb, "NewPermitsResidential", "Issued >= timestamp '2018-01-01 00:00:00' And Classification IN ('COMMERCIAL NEW MULTI-FAMILY', 'RESIDENTIAL NEW DUPLEX', 'RESIDENTIAL NEW SINGLE FAMILY')", '#', '')
arcpy.analysis.SpatialJoin("Lawrence Zoning", filegdb+r"\PlatsCopyPoints", filegdb+r"\PlatsZoning", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.Dissolve("PlatsZoning", filegdb+r"\Subdivisions", "SUBDIVISION;NAME;Zoning", "Join_Count SUM;LotBlock COUNT", "MULTI_PART", "DISSOLVE_LINES")
arcpy.analysis.SpatialJoin("Lawrence Zoning", filegdb+r"\PlatsCopyPoints", filegdb+r"\PlatsZoning", "JOIN_ONE_TO_ONE", "KEEP_ALL")
arcpy.management.Dissolve("PlatsZoning", filegdb+r"\Subdivisions", "SUBDIVISION;NAME;Zoning", "Join_Count SUM;LotBlock COUNT", "MULTI_PART", "DISSOLVE_LINES")
arcpy.analysis.SummarizeWithin(filegdb+r"\Subdivisions", filegdb+r"\ParcelPoints", filegdb+r"\Subdivisions_ParcelsWithin", "KEEP_ALL", "SYSCALACRES Sum", "ADD_SHAPE_SUM", '', None, "NO_MIN_MAJ", "NO_PERCENT")
arcpy.management.AlterField(filegdb+r"\Subdivisions_ParcelsWithin", "SUM_Join_Count", "PlattedLots", "PlattedLots", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.AlterField("Subdivisions_ParcelsWithin", "SUM_SYSCALACRES", "ParceledAcres", "ParceledAcres", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")
arcpy.analysis.SummarizeWithin(filegdb+r"\Subdivisions_ParcelsWithin", filegdb+r"\LandUsePoints", filegdb+r"\Subdivisions_Landuse", "KEEP_ALL", "SYSCALACRES Sum;FinalLand Sum;FinalBuilding Sum;FinalTotal Sum;flivunits Sum;Resyearbuilt Min;Resyearbuilt Max;Resyearbuilt Mean;SUM_Area_SQUAREFEET Sum;SUM_Area_SQUAREFEET Mean;FinalLand Mean;FinalBuilding Mean;FinalTotal Mean;flivunits Mean;FinalLand Stddev;FinalBuilding Stddev;FinalTotal Stddev;flivunits Stddev;Resyearbuilt Stddev", "ADD_SHAPE_SUM", '', None, "NO_MIN_MAJ", "NO_PERCENT", None)

arcpy.conversion.FeatureClassToFeatureClass("LandUsePoints", filegdb, "PermittedLandUseUnbuilt", "Permit IS NOT NULL And (Resyearbuilt IS NULL Or Generalization = 'Vacant')", '#', '')
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "HousingUnits", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'RESIDENTIAL NEW DUPLEX'", None)
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "HousingUnits", "2", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'RESIDENTIAL NEW SINGLE FAMILY'", None)
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "HousingUnits", "1", "PYTHON3", '', "TEXT")
arcpy.management.SelectLayerByAttribute("PermittedLandUseUnbuilt", "NEW_SELECTION", "Classification = 'COMMERCIAL NEW MULTI-FAMILY'", None)
#arcpy.management.CalculateField("PermittedLandUseUnbuilt", "HousingUnits", "str(!Description![0:3]).isdigit()", "PYTHON3", '', "TEXT")
#just reviewed and edited these two, cant figure out how to extract count from description text

arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "PermitYearIssued", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.AddField( filegdb+r"\PermittedLandUseUnbuilt", "PermitYearSubmitted", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PermitYearSubmitted", "str(!Submitted!)[:4]", "PYTHON3", '', "TEXT")
arcpy.management.CalculateField("PermittedLandUseUnbuilt", "PermitYearIssued", "str(!Issued!)[:4]", "PYTHON3", '', "TEXT")
arcpy.analysis.SummarizeWithin( filegdb+r"\Subdivisions_Landuse",  filegdb+r"\PermittedLandUseUnbuilt",  filegdb+r"\Subdivisions", "KEEP_ALL", "SYSCALACRES Sum;Join_Count Sum;HousingUnits Sum;HousingUnits Mean;PermitYearSubmitted Min;PermitYearSubmitted Max;PermitYearIssued Min;PermitYearIssued Max;PermitYearIssued Mean;PermitYearSubmitted Mean", "ADD_SHAPE_SUM", '', None, "NO_MIN_MAJ", "NO_PERCENT", None)

<Result 'PermittedLandUseUnbuilt'>